In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 34.6 MB/s 
     |████████████████████████████████| 6.6 MB 58.0 MB/s 
     |████████████████████████████████| 120 kB 69.6 MB/s 


In [2]:
!pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 31.6 MB/s 


In [3]:
!pip install nltk
import nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random 

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [6]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.tokenize import word_tokenize

factory = StemmerFactory()
stemmer = factory.create_stemmer()
factory2 = StopWordRemoverFactory()
stopword = factory2.create_stop_word_remover()

df = pd.DataFrame({'text':str(), 'label':int()}, index = [])
with open('data_worthcheck/dev.csv') as f:
  for line in f.readlines():
    split = line.split(',')
    split[0]  = stemmer.stem(split[0])
    split[0] = split[0].lower().strip()
    split[0] = stopword.remove(split[0])

    df = df.append({'text': split[0],
                    'label': 1 if split[1] == 'yes' else 0},
                    ignore_index = True)
    
df.drop(index=df.index[0], 
        axis=0, 
        inplace=True)

df.head()

,text,label
1,jek dajal ga depok bang,0
2,detikcom untung depok masuk wilayah nya ridwan...,0
3,df dom jakarta depok yg gunain vc cabang nya c...,0
4,your2rl depok jkt,0
5,doa indonesia selamat virus corona pkb depok g...,0


In [7]:
text = df.text.values
labels = df.label.values

In [8]:
from transformers import AutoTokenizer, AutoModel
from nltk.tokenize import word_tokenize 

tokenizer = AutoTokenizer.from_pretrained("indolem/indobert-base-uncased")

Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/234k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [10]:
def print_rand_sentence():
  '''Displays the tokens and respective IDs of a random text sample'''
  index = random.randint(0, len(text)-1)
  table = np.array([tokenizer.tokenize(text[index]), 
                    tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text[index]))]).T
  print(tabulate(table,
                 headers = ['Tokens', 'Token IDs'],
                 tablefmt = 'fancy_grid'))

print_rand_sentence()

╒═══════════╤═════════════╕
│ Tokens    │   Token IDs │
╞═══════════╪═════════════╡
│ hope      │       25231 │
├───────────┼─────────────┤
│ ##ful     │       12512 │
├───────────┼─────────────┤
│ ##ly      │        2979 │
├───────────┼─────────────┤
│ no        │        2930 │
├───────────┼─────────────┤
│ liter     │        5946 │
├───────────┼─────────────┤
│ ##all     │        3472 │
├───────────┼─────────────┤
│ ##y       │         950 │
├───────────┼─────────────┤
│ sm        │        2829 │
├───────────┼─────────────┤
│ ##g       │         938 │
├───────────┼─────────────┤
│ yg        │        3798 │
├───────────┼─────────────┤
│ infeksi   │        8170 │
├───────────┼─────────────┤
│ cor       │        4219 │
├───────────┼─────────────┤
│ ##ona     │        3559 │
├───────────┼─────────────┤
│ indonesia │        1718 │
├───────────┼─────────────┤
│ kurang    │        2677 │
╘═══════════╧═════════════╛


In [11]:
token_id = []
attention_masks = []

def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  return tokenizer.encode_plus(
                        input_text,
                        add_special_tokens = True,
                        max_length = 32,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )


for sample in text:
  encoding_dict = preprocessing(sample, tokenizer)
  token_id.append(encoding_dict['input_ids']) 
  attention_masks.append(encoding_dict['attention_mask'])


token_id = torch.cat(token_id, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.tensor(labels)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2308: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [12]:
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import matplotlib.pyplot as plt

def print_rand_sentence_encoding():
  '''Displays tokens, token IDs and attention mask of a random text sample'''
  index = random.randint(0, len(text) - 1)
  tokens = tokenizer.tokenize(tokenizer.decode(token_id[index]))
  token_ids = [i.numpy() for i in token_id[index]]
  attention = [i.numpy() for i in attention_masks[index]]

  table = np.array([tokens, token_ids, attention]).T
  print(tabulate(table, 
                 headers = ['Tokens', 'Token IDs', 'Attention Mask'],
                 tablefmt = 'fancy_grid'))
  # np.array[:,1]
  # token_result = nltk.FreqDist(np.array[:,1])
  # print(token_result.most_common())

  # token_result.plot(30,cumulative=False)
  # plt.show()

print_rand_sentence_encoding()

╒══════════╤═════════════╤══════════════════╕
│ Tokens   │   Token IDs │   Attention Mask │
╞══════════╪═════════════╪══════════════════╡
│ [CLS]    │           3 │                1 │
├──────────┼─────────────┼──────────────────┤
│ depok    │        7324 │                1 │
├──────────┼─────────────┼──────────────────┤
│ no       │        2930 │                1 │
├──────────┼─────────────┼──────────────────┤
│ wonder   │       26076 │                1 │
├──────────┼─────────────┼──────────────────┤
│ dah      │        3323 │                1 │
├──────────┼─────────────┼──────────────────┤
│ sk       │        2554 │                1 │
├──────────┼─────────────┼──────────────────┤
│ ##r      │         934 │                1 │
├──────────┼─────────────┼──────────────────┤
│ [SEP]    │           4 │                1 │
├──────────┼─────────────┼──────────────────┤
│ [PAD]    │           0 │                0 │
├──────────┼─────────────┼──────────────────┤
│ [PAD]    │           0 │        

In [13]:
train = pd.DataFrame({'id':int(), 'text':str(), 'label':int()}, index = [])
with open('data_worthcheck/train.csv') as f:
  for line in f.readlines():
    split = line.split(',')
    train = train.append({'id': split[0],
                    'text': split[1],
                    'label': 1 if split[2] == 'yes' else 0},
                    ignore_index = True)
    
train.drop(index=train.index[0], 
    axis=0, 
    inplace=True)
    
train.head()

,id,text,label
1,0,betewe buka twitter cuman ngetweet liat home b...,0
2,1,mas piyuuu mugo2 corona tuh mulut tersumpal ma...,0
3,2,e100ss gini buka informasi sejelas nya identit...,0
4,3,neng solo wes ono terduga corona cobo neng ati...,0
5,4,midiahn nii akun gak takut takut nya isu coron...,0


In [14]:
train_text = train.text.values
train_labels = train.label.values

In [15]:
train_token_id = []
train_attention_masks = []

for curr in train_text:
  encoding_dict = preprocessing(curr, tokenizer)
  train_token_id.append(encoding_dict['input_ids']) 
  train_attention_masks.append(encoding_dict['attention_mask'])


train_token_id = torch.cat(train_token_id, dim = 0)
train_attention_masks = torch.cat(train_attention_masks, dim = 0)
train_labels = torch.tensor(train_labels)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2308: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [37]:
val_ratio = 0.2
# Recommended batch size: 16, 32. See: https://arxiv.org/pdf/1810.04805.pdf
batch_size = 16

# Indices of the train and validation splits stratified by labels
train_idx, val_idx = train_test_split(
    np.arange(len(labels)),
    test_size = val_ratio,
    shuffle = True,
    stratify = labels)

# Train and validation sets
train_set = TensorDataset(train_token_id, 
                          train_attention_masks, 
                          train_labels)
val_set = TensorDataset(token_id, 
                        attention_masks, 
                        labels)

# Prepare DataLoader
train_dataloader = DataLoader(
            train_set,
            sampler = RandomSampler(train_set),
            batch_size = batch_size
        )

validation_dataloader = DataLoader(
            val_set,
            sampler = SequentialSampler(val_set),
            batch_size = batch_size
        )

In [38]:
def b_tp(preds, labels):
  '''Returns True Positives (TP): count of correct predictions of actual class 1'''
  return sum([preds == labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_fp(preds, labels):
  '''Returns False Positives (FP): count of wrong predictions of actual class 1'''
  return sum([preds != labels and preds == 1 for preds, labels in zip(preds, labels)])

def b_tn(preds, labels):
  '''Returns True Negatives (TN): count of correct predictions of actual class 0'''
  return sum([preds == labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_fn(preds, labels):
  '''Returns False Negatives (FN): count of wrong predictions of actual class 0'''
  return sum([preds != labels and preds == 0 for preds, labels in zip(preds, labels)])

def b_metrics(preds, labels):
  '''
  Returns the following metrics:
    - accuracy    = (TP + TN) / N
    - precision   = TP / (TP + FP)
    - recall      = TP / (TP + FN)
    - specificity = TN / (TN + FP)
  '''
  preds = np.argmax(preds, axis = 1).flatten()
  labels = labels.flatten()
  tp = b_tp(preds, labels)
  tn = b_tn(preds, labels)
  fp = b_fp(preds, labels)
  fn = b_fn(preds, labels)
  b_accuracy = (tp + tn) / len(labels)
  b_precision = tp / (tp + fp) if (tp + fp) > 0 else 'nan'
  b_recall = tp / (tp + fn) if (tp + fn) > 0 else 'nan'
  b_specificity = tn / (tn + fp) if (tn + fp) > 0 else 'nan'
  return b_accuracy, b_precision, b_recall, b_specificity

In [39]:
# Load the BertForSequenceClassification model
default='bert-base-uncased'
cahya='cahya/bert-base-indonesian-522M'
cahya2 = "cahya/distilbert-base-indonesian" 
indolem = 'indolem/indobert-base-uncased'
model = BertForSequenceClassification.from_pretrained(indolem, 
                                  num_labels = 2,
                                  output_attentions = False,
                                  output_hidden_states = False,)

# Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5. See: https://arxiv.org/pdf/1810.04805.pdf
optimizer = torch.optim.AdamW(model.parameters(), 
                              lr = 5e-5,
                              eps = 1e-08
                              )

# Run on GPU
model.cuda()

Some weights of the model checkpoint at indolem/indobert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indolem/indober

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31923, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [40]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Recommended number of epochs: 2, 3, 4. See: https://arxiv.org/pdf/1810.04805.pdf
epochs = 4

for _ in trange(epochs, desc = 'Epoch'):
    
    # ========== Training ==========
    
    # Set model to training mode
    model.train()
    
    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        # Forward pass
        train_output = model(b_input_ids, 
                             token_type_ids = None, 
                             attention_mask = b_input_mask, 
                             labels = b_labels)
        # Backward pass
        train_output.loss.backward()
        optimizer.step()
        # Update tracking variables
        tr_loss += train_output.loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    # ========== Validation ==========

    # Set model to evaluation mode
    model.eval()

    # Tracking variables 
    val_accuracy = []
    val_precision = []
    val_recall = []
    val_specificity = []

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
          # Forward pass
          eval_output = model(b_input_ids, 
                              token_type_ids = None, 
                              attention_mask = b_input_mask)
        logits = eval_output.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate validation metrics
        b_accuracy, b_precision, b_recall, b_specificity = b_metrics(logits, label_ids)
        val_accuracy.append(b_accuracy)
        # Update precision only when (tp + fp) !=0; ignore nan
        if b_precision != 'nan': val_precision.append(b_precision)
        # Update recall only when (tp + fn) !=0; ignore nan
        if b_recall != 'nan': val_recall.append(b_recall)
        # Update specificity only when (tn + fp) !=0; ignore nan
        if b_specificity != 'nan': val_specificity.append(b_specificity)

    # print('\n\t - Train loss: {:.4f}'.format(tr_loss / nb_tr_steps))
    # print('\t - Validation Accuracy: {:.4f}'.format(sum(val_accuracy)/len(val_accuracy)))
    # print('\t - Validation Precision: {:.4f}'.format(sum(val_precision)/len(val_precision)) if len(val_precision)>0 else '\t - Validation Precision: NaN')
    # print('\t - Validation Recall: {:.4f}'.format(sum(val_recall)/len(val_recall)) if len(val_recall)>0 else '\t - Validation Recall: NaN')
    # print('\t - Validation Specificity: {:.4f}\n'.format(sum(val_specificity)/len(val_specificity)) if len(val_specificity)>0 else '\t - Validation Specificity: NaN')

Epoch:  25%|██▌       | 1/4 [02:58<08:55, 178.41s/it]


	 - Train loss: 0.0009
	 - Validation Accuracy: 1.0000
	 - Validation Precision: NaN
	 - Validation Recall: NaN
	 - Validation Specificity: 1.0000



Epoch:  50%|█████     | 2/4 [05:53<05:53, 176.71s/it]


	 - Train loss: 0.0000
	 - Validation Accuracy: 1.0000
	 - Validation Precision: NaN
	 - Validation Recall: NaN
	 - Validation Specificity: 1.0000



Epoch:  75%|███████▌  | 3/4 [08:49<02:56, 176.07s/it]


	 - Train loss: 0.0000
	 - Validation Accuracy: 1.0000
	 - Validation Precision: NaN
	 - Validation Recall: NaN
	 - Validation Specificity: 1.0000



Epoch: 100%|██████████| 4/4 [11:44<00:00, 176.03s/it]


	 - Train loss: 0.0000
	 - Validation Accuracy: 1.0000
	 - Validation Precision: NaN
	 - Validation Recall: NaN
	 - Validation Specificity: 1.0000



In [41]:
from sklearn.metrics import classification_report

# new_sentence = 'WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.'
new_sentence = 'sungguh depok keras
# We need Token IDs and Attention Mask for inference on the new sentence
test_ids = []
test_attention_mask = []

# Apply the tokenizer
encoding = preprocessing(new_sentence, tokenizer)

# Extract IDs and Attention Mask
test_ids.append(encoding['input_ids'])
test_attention_mask.append(encoding['attention_mask'])
test_ids = torch.cat(test_ids, dim = 0)
test_attention_mask = torch.cat(test_attention_mask, dim = 0)

# Forward pass, calculate logit predictions
with torch.no_grad():
  output = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

# print(np.argmax(output.logits.cpu().numpy()).flatten().item())
prediction = 'yes' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'no'

print('Input Sentence: ', new_sentence)
print('Predicted Class: ', prediction)

Input Sentence:  warga depok terganggu isu corona
Predicted Class:  no


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2308: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
